In [1]:
import warnings

import pandas as pd

from MLWrappers import SklearnBlackBox
from PrivacyAttacks import MiaPrivacyAttack, AloaPrivacyAttack
from MLPrivacyEvaluator import PrivacyEvaluator

warnings.simplefilter("ignore", UserWarning)

In [2]:
DS_NAME = 'gaussian'
DATA_FOLDER = f'./data/{DS_NAME}'

# we load the target black box model using our wrapper
target = SklearnBlackBox(f'./models/rf_{DS_NAME}.pkl')

# We load the data used to train, test of the model, as well as the shadow data
train_set = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_original_train_set.csv', skipinitialspace=True)[:800]
test_set = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_original_test_set.csv', skipinitialspace=True)[:400]
shadow_set = pd.read_csv(f'{DATA_FOLDER}/{DS_NAME}_shadow_set.csv', skipinitialspace=True)[:500]

In [3]:
# We initialise the attacks, with the desired parameters for each
mia = MiaPrivacyAttack(target, n_shadow_models=5)
aloa = AloaPrivacyAttack(target, n_shadow_models=1, n_noise_samples_fit=200,
                         shadow_test_size=0.3, undersample_attack_dataset=True)
attacks = [mia, aloa]

In [4]:
# We initialise the PrivacyEvaluator object
# We pass the target model and the attacks we want to use
evaluator = PrivacyEvaluator(target, attacks)

In [5]:
# We use the fit() method to execute the attacks, starting from the shadow data
evaluator.fit(shadow_set)

Each shadow model uses 100.000 % of the data


100%|██████████| 500/500 [00:08<00:00, 61.17it/s]

[2.    1.    0.995 0.99  0.985 0.98  0.975 0.97  0.96  0.95  0.93  0.665
 0.58  0.525]


In [6]:
# Then we can obtain the performances using the report() method
results = evaluator.report(train_set, test_set)
print(results)


100%|██████████| 1200/1200 [00:19<00:00, 62.91it/s]

{'mia_attack': {'classification_report': {'IN': {'precision': 0.6591760299625468, 'recall': 0.66, 'f1-score': 0.6595877576514678, 'support': 800}, 'OUT': {'precision': 0.3182957393483709, 'recall': 0.3175, 'f1-score': 0.31789737171464333, 'support': 400}, 'accuracy': 0.5458333333333333, 'macro avg': {'precision': 0.4887358846554588, 'recall': 0.48875, 'f1-score': 0.48874256468305555, 'support': 1200}, 'weighted avg': {'precision': 0.5455492664244882, 'recall': 0.5458333333333333, 'f1-score': 0.545690962339193, 'support': 1200}}}, 'aloa_attack': {'classification_report': {'IN': {'precision': 0.6739702015775635, 'recall': 0.96125, 'f1-score': 0.7923750643997939, 'support': 800}, 'OUT': {'precision': 0.4745762711864407, 'recall': 0.07, 'f1-score': 0.12200435729847496, 'support': 400}, 'accuracy': 0.6641666666666667, 'macro avg': {'precision': 0.5742732363820021, 'recall': 0.515625, 'f1-score': 0.4571897108491344, 'support': 1200}, 'weighted avg': {'precision': 0.6075055581138559, 'recall'

In [7]:
results['mia_attack']

{'classification_report': {'IN': {'precision': 0.6591760299625468,
   'recall': 0.66,
   'f1-score': 0.6595877576514678,
   'support': 800},
  'OUT': {'precision': 0.3182957393483709,
   'recall': 0.3175,
   'f1-score': 0.31789737171464333,
   'support': 400},
  'accuracy': 0.5458333333333333,
  'macro avg': {'precision': 0.4887358846554588,
   'recall': 0.48875,
   'f1-score': 0.48874256468305555,
   'support': 1200},
  'weighted avg': {'precision': 0.5455492664244882,
   'recall': 0.5458333333333333,
   'f1-score': 0.545690962339193,
   'support': 1200}}}

In [8]:
results['aloa_attack']['classification_report']

{'IN': {'precision': 0.6739702015775635,
  'recall': 0.96125,
  'f1-score': 0.7923750643997939,
  'support': 800},
 'OUT': {'precision': 0.4745762711864407,
  'recall': 0.07,
  'f1-score': 0.12200435729847496,
  'support': 400},
 'accuracy': 0.6641666666666667,
 'macro avg': {'precision': 0.5742732363820021,
  'recall': 0.515625,
  'f1-score': 0.4571897108491344,
  'support': 1200},
 'weighted avg': {'precision': 0.6075055581138559,
  'recall': 0.6641666666666667,
  'f1-score': 0.5689181620326875,
  'support': 1200}}

In [9]:
aloa.attack_model.threshold

0.99